In [1]:
import json
import requests
import numpy as np
import pandas as pd
import os
station_df = pd.read_csv("station_df.csv")

# Foursquare

Function to get business data response from Foursquare providing latitude and longitude

In [2]:
FoursquareAPIKey = os.environ["FoursquareAPIKey"]
YelpAPIKey = os.environ["YelpAPIKey"]

In [3]:
def foursquare_search(latitude,longitude):
    url = "https://api.foursquare.com/v3/places/search?ll=" + str(latitude) + "%2C" + str(longitude) + "&radius=1000&categories=13000"

    headers = {
        "accept": "application/json",
        "Authorization": FoursquareAPIKey
    }
    response = requests.get(url, headers=headers)
    return response

Function to get businesses' rating and price response from Foursquare providing business id

In [4]:
def foursquare_detail(fsq_id):
    url = "https://api.foursquare.com/v3/places/" + fsq_id + "?fields=rating%2Cprice"
    headers = {
        "accept": "application/json",
        "Authorization": FoursquareAPIKey
    }
    response = requests.get(url, headers=headers)
    return response

Request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
#define final Foursquare dataframe
foursquare_df = pd.DataFrame()
#to limit the number of API call in order for the local computer to run the code
limit = 50
for index in range(len(station_df)):
    #to limit the number of API call in order for the local computer to run the code
    limit -= 1
    if limit < 0: break
    #call latitude and longitude from citybike.csv
    try:
        latitude = station_df['latitude'][index]
        longitude = station_df['longitude'][index]
         #call API request from Foursquare to get restaurants data
        responseSearch = foursquare_search(latitude,longitude)  
        #parse through response to get restaurants id and name
        business_list = responseSearch.json()['results']
        neededData = ['fsq_id','name','distance','location.address']
        temp_df = pd.json_normalize(business_list)[neededData]
        #adding station data to the temporary dataframe 
        temp_df['station latitude'] = latitude
        temp_df['station longitude'] = longitude
        temp_df['number of bikes'] = station_df['number of bikes'][index]
        temp_df['station id'] = station_df['station id'][index].astype(int)
        #add parsed data to Foursquare dataframe
        foursquare_df = pd.concat([foursquare_df, temp_df], axis=0,ignore_index=True)
    except:
        continue

Call API request to get price and rating for all business in foursquare dataframe and add those data to foursquare dataframe

In [6]:
price = []
rating = []
for index in range(len(foursquare_df)):
    responseDetail = foursquare_detail(foursquare_df['fsq_id'][index])
    try:
        price.append(responseDetail.json()['price'])
    except:
        price.append(np.nan)
    try:
        rating.append(responseDetail.json()['rating'])
    except:
        rating.append(np.nan)

In [7]:
foursquare_df['price'] = price
foursquare_df['rating'] = rating

# Yelp

Function to get response from Yelp providing latitude and longitude

In [8]:
def yelp_request(latitude,longitude):
    url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&term=restaurants&radius=1000&sort_by=best_match"

    headers = {
        "accept": "application/json",
        "Authorization": "Bearer " + YelpAPIKey
    }
    response = requests.get(url, headers=headers)
    return response

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [9]:
#define final Yelp dataframe
yelp_df = pd.DataFrame()
#to limit the number of API call in order for the local computer to run the code
limit = 50
for index in range(len(station_df)):
    try:
        #to limit the number of API call in order for the local computer to run the code
        limit -= 1
        if limit < 0: break
        #call latitude and longitude from citybike.csv
        latitude = station_df['latitude'][index]
        longitude = station_df['longitude'][index]
        #call API request from Yelp to get restaurants data
        response = yelp_request(latitude,longitude)
        #parse through response to get restaurants informations
        business_list = response.json()['businesses']
        neededData = ['name','review_count','rating','distance','price','location.address1']
        temp_df = pd.json_normalize(business_list)[neededData]
         #adding station data to the temporary dataframe 
        temp_df['station latitude'] = latitude
        temp_df['station longitude'] = longitude
        temp_df['number of bikes'] = station_df['number of bikes'][index]
        temp_df['station id'] = station_df['station id'][index].astype(int)
        #add parsed data to Yelp dataframe
        yelp_df = pd.concat([yelp_df, temp_df], axis=0, ignore_index=True)
    except:
        continue

Reformat price and rating data for Yelp dataframe

In [10]:
yelp_df['price'] = yelp_df['price'].replace('$',1).replace('$$',2).replace('$$$',3).replace('$$$$',4)
yelp_df['rating'] = yelp_df['rating']*2

Save dataframes to csv files

In [11]:
foursquare_df.to_csv("foursquare_df.csv", index = False) 

In [12]:
yelp_df.to_csv("yelp_df.csv", index = False) 

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API provides more data about the restaurants than Foursquare API . For the same number of stations, Yelp API usually provide more line of data than the Foursquare API.

Get the top 10 restaurants according to their rating

In [13]:
foursquare_df.sort_values(by = 'rating',ascending=False)[:10]

,fsq_id,name,distance,location.address,station latitude,station longitude,number of bikes,station id,price,rating
450,5329cfc411d2ac08fecb088f,Italian Store,633,5837 Washington Blvd,38.880992,-77.135271,8.0,48,2.0,9.4
353,5e175e1c20e1b00008689d2d,Paris Baguette,916,8344 Broad St,38.928919,-77.225394,1.0,38,1.0,9.2
214,5e175e1c20e1b00008689d2d,Paris Baguette,218,8344 Broad St,38.922450,-77.230340,1.0,24,1.0,9.2
255,546f993e498efe71ca2acd25,Barcelona Wine Bar,254,12023 Town Square St,38.959633,-77.358741,6.0,28,2.0,9.1
278,546f993e498efe71ca2acd25,Barcelona Wine Bar,681,12023 Town Square St,38.960084,-77.353414,0.0,30,2.0,9.1
308,546f993e498efe71ca2acd25,Barcelona Wine Bar,859,12023 Town Square St,38.955079,-77.351649,5.0,33,2.0,9.1
284,546f993e498efe71ca2acd25,Barcelona Wine Bar,147,12023 Town Square St,38.957037,-77.359718,8.0,31,2.0,9.1
245,546f993e498efe71ca2acd25,Barcelona Wine Bar,409,12023 Town Square St,38.955340,-77.357611,3.0,27,2.0,9.1
17,57f54f01498efb4067677e58,Commonwealth Joe Coffee Roasters,569,520 12th St S,38.865553,-77.050030,8.0,3,1.0,9.1
82,524f505f498e802a4fd04854,Kaldi's Social House,604,918 Silver Spring Ave,38.997033,-77.025608,9.0,9,2.0,9.1


In [14]:
yelp_df.sort_values(by = ['rating','review_count','number of bikes'],ascending=False)[:10]

,name,review_count,rating,distance,price,location.address1,station latitude,station longitude,number of bikes,station id
48,Paramount Cafe,338,10.0,639.746063,1.0,1425 S Eads St,38.865553,-77.050030,8.0,3
118,Slider Rider,80,10.0,877.502944,1.0,8600 Zetts Ave,39.114688,-77.171487,9.0,6
97,Taste of Eurasia,68,10.0,732.028379,2.0,428 A Hume Ave,38.833077,-77.059821,11.0,5
219,Delhi Spice,46,10.0,608.155642,2.0,4925 Bethesda Ave,38.977093,-77.094589,2.0,12
113,Taqueria Andrea’s Grill,34,10.0,734.031032,NaN,15701 Frederick Rd,39.114688,-77.171487,9.0,6
335,Old Hat Bar,33,10.0,354.507650,NaN,112 N Saint Asaph St,38.805317,-77.049883,15.0,17
275,Old Hat Bar,33,10.0,617.792923,NaN,112 N Saint Asaph St,38.805648,-77.052930,13.0,14
315,Old Hat Bar,33,10.0,230.196276,NaN,112 N Saint Asaph St,38.804718,-77.043363,6.0,16
296,Old Hat Bar,33,10.0,1174.103863,NaN,112 N Saint Asaph St,38.814577,-77.052808,2.0,15
33,Cheros Grill,24,10.0,1111.918761,NaN,7224 Carroll Ave,38.986743,-77.000035,7.0,2
